In [27]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
#importando dados
dados_api = requests.get('https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json')
resultado_dados_api = dados_api.json()
df = pd.json_normalize(resultado_dados_api)
#separando colunas importantes
series_importantes = ['Churn', 'customer.gender', 'customer.SeniorCitizen', 'customer.tenure', 
                      'phone.PhoneService', 'internet.InternetService', 'account.Contract', 'account.PaymentMethod', 
                      'account.Charges.Monthly', 'account.Charges.Total']
df = df[series_importantes]
#tratando dados 
remover = df.query('Churn == ""').index
df.drop(remover, axis=0, inplace=True)
df['customer.SeniorCitizen'] = df['customer.SeniorCitizen'].astype(str)
df['customer.SeniorCitizen'] = df['customer.SeniorCitizen'].map({'0': 'No', '1': 'Yes'})
#normalizando os tipos de dados
col_string = ['Churn', 'customer.gender', 'customer.SeniorCitizen', 'phone.PhoneService', 'internet.InternetService',
              'account.Contract', 'account.PaymentMethod']
df = df.astype({col: 'string' for col in col_string})
df['account.Charges.Total'] = df['account.Charges.Total'].replace(' ', '0')
df['account.Charges.Total'] = df['account.Charges.Total'].astype(np.float64)
#criando contas_diarias mantendo a padronização no nome
df['account.Charges.Daily'] = (df['account.Charges.Monthly']/30).round(2)
df.head()

,Churn,customer.gender,customer.SeniorCitizen,customer.tenure,phone.PhoneService,internet.InternetService,account.Contract,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,account.Charges.Daily
0,No,Female,No,9,Yes,DSL,One year,Mailed check,65.6,593.30,2.19
1,No,Male,No,9,Yes,DSL,Month-to-month,Mailed check,59.9,542.40,2.00
2,Yes,Male,No,4,Yes,Fiber optic,Month-to-month,Electronic check,73.9,280.85,2.46
3,Yes,Male,Yes,13,Yes,Fiber optic,Month-to-month,Electronic check,98.0,1237.85,3.27
4,Yes,Female,Yes,3,Yes,Fiber optic,Month-to-month,Mailed check,83.9,267.40,2.80


**Distribuição da evasão**

In [60]:
churn_counts = df['Churn'].value_counts().reset_index()
churn_counts.columns = ['Churn', 'Count']

fig = px.bar(churn_counts, y='Churn', x='Count', title='Número de pessoas que optaram ou não pela evasão', color='Churn', color_discrete_sequence=px.colors.qualitative.Pastel,  labels={'Churn': 'Evasão', 'count': 'Número de pessoas'})
fig.show()

**Evasão por gênero**

In [77]:
df_gender = df[['customer.gender', 'Churn']].value_counts().reset_index()
df_gender_yes = df_gender.query('Churn == "Yes"')

fig = px.pie(df_gender_yes, values='count', names='customer.gender', title='Número de evasão entre homens e mulheres', color='customer.gender', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(
    width = 900,
    height= 500,
    title={
        'text': 'Número de evasão entre homens e mulheres',
        'y':0.95,                     
        'x':0.5,                        
        'xanchor': 'center',            
        'yanchor': 'top'  
    })
fig.update_traces(textposition='inside', textinfo='percent+label', rotation=45, hovertemplate="<b>%{label}</b><br>Total: %{value}<br>Porcentagem: %{percent}")
fig.show()


In [35]:
# df_gender_no = df_gender.query('Churn == "No"')
# fig = px.bar(df_gender_no, y='customer.gender', x='count', title='Número de pessoas que não cancelaram entre homens e mulheres', color='customer.gender', color_discrete_sequence=px.colors.qualitative.D3)
# fig.update_layout(xaxis_title='Número de pessoas', yaxis_title='Gêneros')
# fig.show()

**Evasão por idade**

In [76]:
df_senior = df[['Churn', 'customer.SeniorCitizen']].value_counts().reset_index()
df_senior_yes = df_senior.query('Churn == "Yes"')
fig = px.pie(df_senior_yes, values='count', names='customer.SeniorCitizen', title='Número de evasão entre clientes +60 (No/Yes)', color='customer.SeniorCitizen', color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(
    width = 900,
    height= 500,
    title={
        'text': 'Número de evasão entre clientes +60 (No/Yes)',
        'y':0.95,                     
        'x':0.5,                        
        'xanchor': 'center',            
        'yanchor': 'top'  
    })

fig.update_traces(textposition='inside', textinfo='percent+label', rotation=45, hovertemplate="<b>%{label}</b><br>Total: %{value}<br>Porcentagem: %{percent}")
fig.show()


Mais de 50% das pessoas fazendo evasão não são +60

**Evasão por tipo de pagameto**

In [58]:
df_pagamentos = df[['Churn', 'account.PaymentMethod']].value_counts().reset_index()
df_pagamentos = df_pagamentos.query('Churn == "Yes"')

fig = px.bar(df_pagamentos, y='account.PaymentMethod', x='count', color='account.PaymentMethod', color_discrete_sequence=px.colors.qualitative.D3, labels={'account.PaymentMethod': 'Tipo de pagamento', 'count': 'Número de pessoas'})
fig.show()

O tipo de pagamento com maio evasão é o de pagamento eletrônico

In [57]:
df_tipo_contrato = df[['Churn', 'account.Contract']].value_counts().reset_index()
df_tipo_contrato = df_tipo_contrato.query('Churn == "Yes"')

fig = px.bar(df_tipo_contrato, x='count', y='account.Contract', title='Taxa de evasão por tipo de contrato', color='account.Contract', color_discrete_sequence=px.colors.qualitative.Dark2, labels={'account.Contract': 'Tipo de contrato', 'count': 'Número de pessoas'})
fig.show()

O tipo de contrato com maior evasão é o mensal